In [14]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [15]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [16]:
db = SQLDatabase.from_uri("sqlite:////Users/altaf/Projects/LLM/LangChain/Chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [17]:
# agent_executor = create_sql_agent(
#     llm=OpenAI(temperature=0),
#     toolkit=toolkit,
#     verbose=True,
#     agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
# )

In [18]:
agent_executor = create_sql_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS
)

In [20]:
agent_executor.run("List of the Artist, Track and Sales Amount")



> Entering new  chain...

Invoking: `sql_db_list_tables` with `{}`


Invoice, Customer, Album, InvoiceLine, Artist, MediaType, PlaylistTrack, Playlist, Employee, Genre, Track
Invoking: `sql_db_schema` with `InvoiceLine, Artist`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("InvoiceLineId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")
)

/*
3 rows from InvoiceLine table:
InvoiceLineId	InvoiceId	TrackId	UnitPrice	Quantity
1	1	2	0.99	1
2	1	4	0.99	1
3	2	6	0.99	1
*/Based on the schema, I can query the "Artist" and "InvoiceLine" tables to get the list of artists, tracks

'Based on the schema, I can query the "Artist" and "InvoiceLine" tables to get the list of artists, tracks, and sales amount. Let me construct the query for you.\n\n```sql\nSELECT Artist.Name AS Artist, Track.Name AS Track, InvoiceLine.UnitPrice * InvoiceLine.Quantity AS SalesAmount\nFROM Artist\nJOIN InvoiceLine ON Artist.ArtistId = InvoiceLine.TrackId\nLIMIT 10;\n```\n\nThis query joins the "Artist" and "InvoiceLine" tables on the "ArtistId" and "TrackId" columns respectively. It selects the artist name, track name, and calculates the sales amount by multiplying the unit price and quantity from the "InvoiceLine" table. The results are limited to 10 rows.\n\nPlease let me know if you would like me to execute this query.'

In [19]:
agent_executor.run(
    "Resolve the database schema and provide a flat structure"
)



> Entering new  chain...

Invoking: `sql_db_list_tables` with ``


Invoice, Customer, Album, InvoiceLine, Artist, MediaType, PlaylistTrack, Playlist, Employee, Genre, Track
Invoking: `sql_db_schema` with `Invoice, Customer, Album, InvoiceLine, Artist, MediaType, PlaylistTrack, Playlist, Employee, Genre, Track`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Employee" (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60), 
	PRIMARY KEY ("EmployeeId"), 
	FOREIGN KEY("ReportsTo") REFERENCES "Employ

'The database schema consists of the following tables:\n\n1. Artist:\n   - ArtistId (INTEGER)\n   - Name (NVARCHAR(120))\n\n2. Employee:\n   - EmployeeId (INTEGER)\n   - LastName (NVARCHAR(20))\n   - FirstName (NVARCHAR(20))\n   - Title (NVARCHAR(30))\n   - ReportsTo (INTEGER)\n   - BirthDate (DATETIME)\n   - HireDate (DATETIME)\n   - Address (NVARCHAR(70))\n   - City (NVARCHAR(40))\n   - State (NVARCHAR(40))\n   - Country (NVARCHAR(40))\n   - PostalCode (NVARCHAR(10))\n   - Phone (NVARCHAR(24))\n   - Fax (NVARCHAR(24))\n   - Email (NVARCHAR(60))\n\n3. Genre:\n   - GenreId (INTEGER)\n   - Name (NVARCHAR(120))\n\n4. MediaType:\n   - MediaTypeId (INTEGER)\n   - Name (NVARCHAR(120))\n\n5. Playlist:\n   - PlaylistId (INTEGER)\n   - Name (NVARCHAR(120))\n\n6. Album:\n   - AlbumId (INTEGER)\n   - Title (NVARCHAR(160))\n   - ArtistId (INTEGER)\n\n7. Customer:\n   - CustomerId (INTEGER)\n   - FirstName (NVARCHAR(40))\n   - LastName (NVARCHAR(20))\n   - Company (NVARCHAR(80))\n   - Address (NVA

In [13]:
agent_executor.run(
    "Provide the Yearly sales amount for each Artist in each Country"
)



> Entering new  chain...

Invoking: `sql_db_list_tables` with ``


Invoice, Customer, Album, InvoiceLine, Artist, MediaType, PlaylistTrack, Playlist, Employee, Genre, Track
Invoking: `sql_db_schema` with `Invoice, Customer, Artist`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 11847 tokens (11579 in the messages, 268 in the functions). Please reduce the length of the messages or functions.